# 12.Recommend Novel Recipes to the user

In [ ]:
import networkx as nx
import pandas as pd
from pandas import DataFrame
from node2vec import Node2Vec
import numpy as np

In [2]:
target_user= 'u2437545'
rating_threshold=3
similarity_threshold_low=float(0.4)
score_check=10
c1=0.2
c2=0.2
c3=0.34
c4=0.00026
recomender_limit=5
ing_change=0

In [3]:
G = nx.read_edgelist("new_recipe-user.txt",create_using=nx.DiGraph())
y_df=pd.read_csv('y_values.csv')
y_df['recipe_id'] = 'r' + y_df['recipe_id'].astype(str)
d=pd.read_csv('no_drop.csv')
x_df=pd.read_csv('Normalized_LR_Nutritional_content.csv')

In [4]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 97861
Number of edges: 676946
Average in degree:   6.9174
Average out degree:   6.9174


In [5]:
import gensim
# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load("new_model_recipe_user")
ing_model= gensim.models.Word2Vec.load("model_ingredients_recipe")

In [6]:
similarity_of_users=model.wv.most_similar(target_user,topn=50)

In [8]:
k=list()
similar_users=list()
for x in similarity_of_users:
    if ('u' in str(x[0])):
        k.append(x)

similarity_of_users=k

for x in similarity_of_users:
    if x[1] >= similarity_threshold_low:
        similar_users.append(x[0])

In [9]:
print(similarity_of_users)

[('u6434901', 0.8243931531906128), ('u4441156', 0.8214802145957947), ('u6173748', 0.7832164168357849), ('u4145595', 0.7767403721809387), ('u4857989', 0.7224301695823669), ('u2492894', 0.6410077810287476), ('u1206350', 0.6213952898979187), ('u7492198', 0.6188927888870239), ('u3722843', 0.6182271242141724), ('u5789670', 0.606837272644043), ('u3366428', 0.586632251739502), ('u2273803', 0.5530738830566406), ('u2493584', 0.5273569822311401), ('u824525', 0.5151377320289612), ('u2719267', 0.5119464993476868), ('u329135', 0.511273980140686), ('u2254026', 0.5041066408157349), ('u688746', 0.4908807575702667), ('u4458934', 0.4832008183002472), ('u3115700', 0.48075056076049805), ('u3313278', 0.47740650177001953), ('u1348232', 0.47298407554626465), ('u3447961', 0.46782612800598145), ('u2414000', 0.4642663300037384), ('u2956616', 0.46091362833976746), ('u2065679', 0.4580599069595337), ('u177808', 0.45796170830726624), ('u3138802', 0.45510321855545044), ('u1542543', 0.4505976140499115), ('u4757855', 

In [10]:
print(similar_users)

['u6434901', 'u4441156', 'u6173748', 'u4145595', 'u4857989', 'u2492894', 'u1206350', 'u7492198', 'u3722843', 'u5789670', 'u3366428', 'u2273803', 'u2493584', 'u824525', 'u2719267', 'u329135', 'u2254026', 'u688746', 'u4458934', 'u3115700', 'u3313278', 'u1348232', 'u3447961', 'u2414000', 'u2956616', 'u2065679', 'u177808', 'u3138802', 'u1542543', 'u4757855', 'u3429508', 'u6292138', 'u987674', 'u4048263', 'u539252', 'u1045421', 'u2465853', 'u483206', 'u1516436', 'u3389252', 'u1005679', 'u2651494', 'u5068637', 'u1038075', 'u2364221', 'u20408224']


In [11]:
a=set()
q=nx.edges(G,[target_user])
for x in q:
    a.add(x[1])


In [12]:
print(a)

{'r8938'}


In [13]:
b=set()
similar_user_rating=list()
for s in similar_users:
    for u, v, data in G.out_edges(s, data=True):
        similar_user_rating.append([u, v, data['weight']])
        if data['weight']>rating_threshold:
            b.add(v)


In [14]:
print(similar_user_rating)

[['u6434901', 'r8938', 5], ['u4441156', 'r8938', 5], ['u6173748', 'r8938', 4], ['u4145595', 'r8938', 5], ['u4857989', 'r8938', 5], ['u2492894', 'r18378', 5], ['u2492894', 'r8938', 5], ['u1206350', 'r13010', 5], ['u1206350', 'r8938', 5], ['u7492198', 'r174720', 5], ['u7492198', 'r8938', 5], ['u3722843', 'r8938', 5], ['u3722843', 'r20096', 5], ['u5789670', 'r7193', 4], ['u5789670', 'r8938', 5], ['u5789670', 'r12142', 5], ['u3366428', 'r71634', 5], ['u3366428', 'r69919', 5], ['u3366428', 'r8938', 5], ['u3366428', 'r106630', 5], ['u2273803', 'r9377', 5], ['u2273803', 'r8938', 5], ['u2273803', 'r7159', 5], ['u2493584', 'r10813', 5], ['u2493584', 'r8938', 4], ['u2493584', 'r53314', 5], ['u824525', 'r72392', 5], ['u824525', 'r14565', 5], ['u824525', 'r26699', 4], ['u824525', 'r11966', 5], ['u824525', 'r24083', 5], ['u824525', 'r24087', 5], ['u824525', 'r19037', 5], ['u824525', 'r7443', 5], ['u824525', 'r16431', 4], ['u824525', 'r18031', 5], ['u824525', 'r18033', 5], ['u824525', 'r43914', 5], 

In [15]:
print(b)

{'r146819', 'r8544', 'r14837', 'r141678', 'r21491', 'r11266', 'r18346', 'r76705', 'r14629', 'r86939', 'r23600', 'r23060', 'r14646', 'r10275', 'r17999', 'r10921', 'r216899', 'r10813', 'r82923', 'r25224', 'r53314', 'r22831', 'r19934', 'r45361', 'r14231', 'r12801', 'r11757', 'r14724', 'r10721', 'r24083', 'r51803', 'r23890', 'r16539', 'r15412', 'r8948', 'r8665', 'r69919', 'r25155', 'r10849', 'r39253', 'r88186', 'r49262', 'r22180', 'r143298', 'r8952', 'r85798', 'r84310', 'r20445', 'r7193', 'r13669', 'r21712', 'r26297', 'r16574', 'r8580', 'r13316', 'r87438', 'r72114', 'r13718', 'r19037', 'r46887', 'r11675', 'r186746', 'r34746', 'r12839', 'r23944', 'r17219', 'r31542', 'r11587', 'r49777', 'r40292', 'r59467', 'r90233', 'r8818', 'r15222', 'r11746', 'r213211', 'r15987', 'r13944', 'r143069', 'r21685', 'r9516', 'r80746', 'r12026', 'r23766', 'r16611', 'r41236', 'r15401', 'r22605', 'r59866', 'r36002', 'r18256', 'r16907', 'r8430', 'r16276', 'r69410', 'r16952', 'r149984', 'r25885', 'r13010', 'r21014', 

In [16]:
print(b-a)

{'r146819', 'r8544', 'r14837', 'r106630', 'r13664', 'r141678', 'r84474', 'r77489', 'r21491', 'r24822', 'r11266', 'r14520', 'r14601', 'r18346', 'r15477', 'r76705', 'r8499', 'r177078', 'r64646', 'r14629', 'r59571', 'r25202', 'r86939', 'r18378', 'r23600', 'r74909', 'r76697', 'r8308', 'r23060', 'r25364', 'r14646', 'r10275', 'r25272', 'r50644', 'r17999', 'r10921', 'r216899', 'r11688', 'r10813', 'r82923', 'r25224', 'r40352', 'r53314', 'r9023', 'r22831', 'r185564', 'r50726', 'r14162', 'r10344', 'r13023', 'r19934', 'r9387', 'r45361', 'r14231', 'r12801', 'r11757', 'r45954', 'r8547', 'r82487', 'r14724', 'r8501', 'r8562', 'r18033', 'r17742', 'r54675', 'r15023', 'r72392', 'r10721', 'r24083', 'r86069', 'r51803', 'r23890', 'r16539', 'r15412', 'r8948', 'r21392', 'r12142', 'r56673', 'r11758', 'r16947', 'r8665', 'r8534', 'r69919', 'r25155', 'r10849', 'r17422', 'r82892', 'r39253', 'r84745', 'r88186', 'r39455', 'r14624', 'r18919', 'r8350', 'r13663', 'r16647', 'r21402', 'r49262', 'r69538', 'r22180', 'r247

# RECOMEND HEALTHY

In [17]:
if b-a == set():
    no_health_recomended=list(b)
else:
    no_health_recomended=list(b-a)

In [18]:
print(no_health_recomended)

['r146819', 'r8544', 'r14837', 'r106630', 'r13664', 'r141678', 'r84474', 'r77489', 'r21491', 'r24822', 'r11266', 'r14520', 'r14601', 'r18346', 'r15477', 'r76705', 'r8499', 'r177078', 'r64646', 'r14629', 'r59571', 'r25202', 'r86939', 'r18378', 'r23600', 'r74909', 'r76697', 'r8308', 'r23060', 'r25364', 'r14646', 'r10275', 'r25272', 'r50644', 'r17999', 'r10921', 'r216899', 'r11688', 'r10813', 'r82923', 'r25224', 'r40352', 'r53314', 'r9023', 'r22831', 'r185564', 'r50726', 'r14162', 'r10344', 'r13023', 'r19934', 'r9387', 'r45361', 'r14231', 'r12801', 'r11757', 'r45954', 'r8547', 'r82487', 'r14724', 'r8501', 'r8562', 'r18033', 'r17742', 'r54675', 'r15023', 'r72392', 'r10721', 'r24083', 'r86069', 'r51803', 'r23890', 'r16539', 'r15412', 'r8948', 'r21392', 'r12142', 'r56673', 'r11758', 'r16947', 'r8665', 'r8534', 'r69919', 'r25155', 'r10849', 'r17422', 'r82892', 'r39253', 'r84745', 'r88186', 'r39455', 'r14624', 'r18919', 'r8350', 'r13663', 'r16647', 'r21402', 'r49262', 'r69538', 'r22180', 'r247

In [19]:
for x in no_health_recomended:
    kkff=str(y_df[y_df['recipe_id'] == x]).split()[7:]
    print(kkff)

['2.384148', '4.108659', '1.088289', '75.689']
['0.405241', '20.75568', '31.88243', '256.9474']
['0.218825', '5.3822', '0.55675', '74.615']
['0.518188', '1.507312', '4.928381', '87.47375']
['2.17792', '24.77856', '22.50834', '1021.501']
['21.3203', '5.0388', '21.1669', '990.115']
['0.0', '0.0', '0.0', '0.0']
['1.868682', '31.92233', '24.35863', '106.7278']
['2.413006', '30.17119', '32.80782', '619.4352']
['5.824857', '19.48422', '19.40075', '278.6573']
['15.53165', '10.82377', '3.031693', '98.78986']
['1.528632', '9.053069', '30.05184', '1313.496']
['15.35396', '64.65548', '40.8279', '1875.49']
['4.301215', '3.299654', '8.965934', '270.3102']
['12.08894', '5.834477', '1.793485', '77.50406']
['11.00994', '42.01231', '23.53924', '1550.901']
['2.173177', '63.14806', '44.31533', '402.9607']
['10.96333', '18.10768', '50.16825', '980.2046']
['2.445646', '7.878384', '12.6964', '336.1061']
['10.06505', '50.79366', '48.529', '1549.868']
['4.310316', '3.836229', '4.631779', '560.8732']
['0.72486

['29.01467', '11.16779', '2.932419', '164.7692']
['4.595185', '7.16476', '5.566025', '76.1185']
['5.425535', '29.75189', '17.35936', '2478.249']
['22.56655', '18.12535', '3.862108', '206.6247']
['0.337762', '9.519725', '20.85414', '354.4968']
['1.832045', '6.633124', '28.69913', '401.7416']
['15.56109', '34.73977', '29.54446', '540.4072']
['13.81284', '29.52939', '50.94195', '1899.71']
['2.143464', '34.70129', '15.91584', '505.0821']
['0.924948', '29.11786', '31.7073', '811.3117']
['10.73813', '3.828148', '0.634732', '12.04712']
['0.85127', '23.68352', '45.63106', '638.726']
['18.36369', '20.10193', '18.39987', '763.2519']
['6.559135', '57.12466', '25.09073', '1029.817']
['10.15474', '21.44849', '19.53144', '1105.731']
['3.061418', '12.48757', '26.62463', '952.2217']
['0.323848', '19.16148', '23.19769', '698.2523']
['0.703688', '17.91482', '30.89253', '607.6538']
['0.941913', '2.559506', '6.323143', '437.4094']
['0.894573', '35.31625', '25.02522', '2159.14']
['21.80115', '8.619831', '2

['2.238302', '7.550885', '4.830691', '881.5023']
['0.31', '16.7762', '30.2646', '261.175']
['0.0', '0.0', '0.0', '0.0']
['6.522721', '28.08123', '8.087998', '140.2099']
['4.576086', '14.57136', '19.2706', '720.9535']
['21.15303', '0.2684', '0.598625', '0.99']
['7.378755', '13.36951', '31.44464', '292.957']
['7.520916', '8.32513', '2.279192', '529.3124']
['0.191283', '11.7067', '4.429709', '315.1425']
['41.65023', '29.89823', '5.557654', '476.0265']
['2.74211', '19.37363', '11.0941', '287.2297']
['23.05742', '4.224392', '3.067441', '174.8811']
['1.126391', '17.89084', '33.31707', '786.0479']
['3.555556', '39.96807', '60.32586', '593.5226']
['1.20555', '10.83205', '30.40238', '326.424']
['1.696633', '13.27944', '13.4421', '482.5969']
['0.906413', '11.4762', '5.263068', '167.4228']
['31.81237', '18.92498', '3.268971', '293.3676']
['16.40176', '29.9706', '37.08934', '473.3896']
['49.65675', '15.41938', '4.834639', '385.2637']
['1.093969', '38.2929', '37.37558', '1248.425']
['7.555897', '21

In [20]:
probables=list()
for x in no_health_recomended:
    kkff=str(y_df[y_df['recipe_id'] == x]).split()[7:]
    temp=(c1*float(kkff[0]))+(c2*float(kkff[1]))+(c3*float(kkff[2]))+(c4*float(kkff[3]))
    probables.append([x,temp])

In [21]:
print(probables)

[['r146819', 1.6882588000000003], ['r8544', 15.139016724], ['r14837', 1.3288999000000001], ['r106630', 2.1034927150000002], ['r13664', 13.30972186], ['r141678', 12.7259959], ['r84474', 0.0], ['r77489', 15.067885828], ['r21491', 17.832551152], ['r24822', 11.730521298], ['r11266', 6.3275449836], ['r14520', 12.67547476], ['r14601', 30.371001400000004], ['r18346', 4.638872012000001], ['r15477', 4.2146193556], ['r76705', 19.01102586], ['r8499', 28.236229382000005], ['r177078', 23.126260195999997], ['r64646', 6.468969586000001], ['r14629', 29.07456768], ['r59571', 3.3499408920000002], ['r25202', 10.555983062000001], ['r86939', 13.157623406], ['r18378', 16.626476202], ['r23600', 16.539623080000002], ['r74909', 2.602648116], ['r76697', 6.120729180800001], ['r8308', 7.8523639759999995], ['r23060', 20.849580060000005], ['r25364', 13.450645999999999], ['r14646', 12.049669444], ['r10275', 7.95609596], ['r25272', 4.95693789], ['r50644', 11.2899796], ['r17999', 7.955042992000001], ['r10921', 8.59286

In [22]:
if len(probables)>recomender_limit:
    probables.sort(key=lambda x: float(x[1]),reverse=True)
    probables=probables[:recomender_limit]
    

In [23]:
print(probables)

[['r23314', 45.67953054], ['r174720', 41.37879504], ['r23157', 39.340005244], ['r8534', 38.709409040000004], ['r91456', 37.67226916]]


In [24]:
healthy_recipes_check=list()
for x in probables:
    if x[1] > score_check:
        healthy_recipes_check.append(x)
print(healthy_recipes_check)

[['r23314', 45.67953054], ['r174720', 41.37879504], ['r23157', 39.340005244], ['r8534', 38.709409040000004], ['r91456', 37.67226916]]


In [25]:
check=list()
scores=list()
for x in healthy_recipes_check:
    check.append(x[0])
    scores.append(x[1])

# INGREDIENT LEVEL

In [26]:
d=d.drop(['image_url', 'cooking_directions','nutritions'], axis = 1)
d['recipe_id'] = 'r' + d['recipe_id'].astype(str)

In [27]:
y_df.head()

,recipe_id,sugars,fat,protein,sodium
0,r240488,19.841460,11.675210,36.398780,2606.7640
1,r218939,0.236130,23.625190,23.912650,762.8050
2,r87211,1.974210,29.367610,32.375370,1075.5270
3,r245714,2.452803,7.600288,7.059566,189.8088
4,r218545,8.222667,7.977177,26.120210,1301.8880


In [28]:
dr= pd.DataFrame()
p1=list()
p2=list()
p3=list()
p4=list()
p5=list()
p6=list()
for x in check:
    p1.append("novel_"+d[d['recipe_id'] == x]['recipe_name'].tolist()[0])
    p2.append(d[d['recipe_id'] == x]['ingredients'].tolist()[0])
    p3.append(y_df[y_df['recipe_id'] == x]['sugars'].tolist()[0])
    p4.append(y_df[y_df['recipe_id'] == x]['fat'].tolist()[0])
    p5.append(y_df[y_df['recipe_id'] == x]['protein'].tolist()[0])
    p6.append(y_df[y_df['recipe_id'] == x]['sodium'].tolist()[0])
values=[check,p1,p2,p3,p4,p5,p6,scores]
columns=['recipe_id','recipe_name','ingredients','sugars','fat','protein','sodium','health_score']
for x in range(2):
    dr[columns[x]]=values[x]


In [29]:
print(dr)
print(p2)

  recipe_id                                        recipe_name
0    r23314                         novel_Chicken Chimichangas
1   r174720  novel_Feta and Bacon Stuffed Chicken with Onio...
2    r23157                              novel_Turkey in a Bag
3     r8534                                novel_Bou's Chicken
4    r91456             novel_Good Ole' Southern Frogmore Stew
['chicken broth,longgrain rice,enchilada sauce,onion,flour tortillas,breast,monterey jack cheese,olives,beans,oil,topping,avocados,cilantro,lemon juice,onions,jalapeno chile peppers,tomato,lettuce,cream,cheddar cheese', 'bacon,feta cheese,cream,oregano,ground pepper,skinless boneless breast halves,flour,eggs,bread crumbs,potatoes,onion vidalia,butter,cream', 'turkey,salt pepper,flour,celery,onions', 'mustard,sugar,soy sauce,vinegar,ketchup,water,chicken', 'quarts water,lemon,medium onion,garlic,coarse salt,crab boil,medium potatoes,sausage,ears corn,shrimp,butter']


In [30]:
scores_new=list()
for i in range(len(check)):
    q=list()
    q1=list()
    q2=list()
    q.append(q1)
    q.append(q2)
    temp=list()
    temp1=p2[i].split(',')
    print(temp1)
    temp.append(temp1[ing_change])
    for node, _ in ing_model.most_similar(temp1[ing_change],topn=10):
        # Show only ingredient
        if any(char.isdigit() for char in node) == False:
            temp.append(node)
        if len(temp)==2:
            break
    #print(node,temp)
            
    for j in range(2):
        q[j].append(x_df[x_df['Unique_Ingredients']==temp[j]]['sugars'].tolist()[0])
        q[j].append(x_df[x_df['Unique_Ingredients']==temp[j]]['fat'].tolist()[0])
        q[j].append(x_df[x_df['Unique_Ingredients']==temp[j]]['protein'].tolist()[0])
        q[j].append(x_df[x_df['Unique_Ingredients']==temp[j]]['sodium'].tolist()[0])    
        
    p3[i]=p3[i]+(q[1][0]-q[0][0])#sugar
    p4[i]=p4[i]+(q[1][1]-q[0][1])#fat
    p5[i]=p3[i]+(q[1][2]-q[0][2])#protein
    p6[i]=p3[i]+(q[1][3]-q[0][3])#sodium
    
    if p3[i]<0:
        p3[i]=0
    if p4[i]<0:
        p4[i]=0
    if p5[i]<0:
        p5[i]=0
    if p6[i]<0:
        p6[i]=0
    scores_new.append((c1*float(p3[i]))+(c2*float(p4[i]))+(c3*float(p5[i]))+(c4*float(p6[i])))
    temp1[0]=temp[1]
    p2[i]=",".join(temp1)
dr['ingridients']=p2
dr['sugars']=p3
dr['fat']=p4
dr['protein']=p5
dr['sodium']=p6
dr['scores_old']=scores
dr['scores_new']=scores_new
  
         
    

['chicken broth', 'longgrain rice', 'enchilada sauce', 'onion', 'flour tortillas', 'breast', 'monterey jack cheese', 'olives', 'beans', 'oil', 'topping', 'avocados', 'cilantro', 'lemon juice', 'onions', 'jalapeno chile peppers', 'tomato', 'lettuce', 'cream', 'cheddar cheese']
['bacon', 'feta cheese', 'cream', 'oregano', 'ground pepper', 'skinless boneless breast halves', 'flour', 'eggs', 'bread crumbs', 'potatoes', 'onion vidalia', 'butter', 'cream']
['turkey', 'salt pepper', 'flour', 'celery', 'onions']
['mustard', 'sugar', 'soy sauce', 'vinegar', 'ketchup', 'water', 'chicken']
['quarts water', 'lemon', 'medium onion', 'garlic', 'coarse salt', 'crab boil', 'medium potatoes', 'sausage', 'ears corn', 'shrimp', 'butter']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


In [32]:
dr.to_csv('1Novel_Recomendation_u2437545.csv')